In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
df = pd.read_excel('./Danh sách cấp tỉnh kèm theo quận huyện phường xã.xls')

WARNING *** file size (2573784) not 512 + multiple of sector size (512)


In [4]:
tinh_tp_set = set(df['Tỉnh Thành Phố'])

In [5]:
def spelling_correction(phrase):
    # fix uỵ
    i = phrase.find('uỵ')
    if i != -1:
        phrase = phrase[:i] + 'ụy' + phrase[i+2:]
    i = phrase.find('oà')
    if i != -1 and (i + 2 == len(phrase) or phrase[i + 2].isspace()):
        phrase = phrase[:i] + 'òa' + phrase[i+2:]
    i = phrase.find('uỷ')
    if i != -1:
        phrase = phrase[:i] + 'ủy' + phrase[i+2:]
    return phrase

In [6]:
def tinh_tp_normalize(tinh_tp):
    if not isinstance(tinh_tp, str):
        return np.nan
    if tinh_tp == 'Tỉnh Bà Rịa - Vũng Tàu':
        return 'bà rịa vũng tàu|bà rịa|vũng tàu'
    tinh_tp = spelling_correction(tinh_tp)
    if tinh_tp.startswith('Tỉnh'):
        return tinh_tp[4:].strip().lower()
    elif tinh_tp.startswith('Thành phố'):
        return tinh_tp[9:].strip().lower()
    else:
        return tinh_tp.strip().lower()

In [7]:
df['Tỉnh Thành Phố Normalized'] = df['Tỉnh Thành Phố'].apply(tinh_tp_normalize)

In [8]:
tinh_tp_normalized_set = set()

In [9]:
for tinh_tp in df['Tỉnh Thành Phố Normalized']:
    tinh_tp_normalized_set.add(tinh_tp)

In [10]:
tinh_tp_normalized_set.add('bà rịa')
tinh_tp_normalized_set.add('vũng tàu')
tinh_tp_normalized_set.discard('')

In [11]:
with open('./src/data/provinces.txt', 'w') as f:
    for tinh_tp in tinh_tp_normalized_set:
        f.write(tinh_tp + '\n')

In [12]:
df.head()

,Tỉnh Thành Phố,Mã TP,Quận Huyện,Mã QH,Phường Xã,Mã PX,Cấp,Tên Tiếng Anh,Tỉnh Thành Phố Normalized
0,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Phúc Xá,1,Phường,NaN,hà nội
1,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Trúc Bạch,4,Phường,NaN,hà nội
2,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Vĩnh Phúc,6,Phường,Vinh Phuc Commune,hà nội
3,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Cống Vị,7,Phường,NaN,hà nội
4,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Liễu Giai,8,Phường,Lieu Giai Commune,hà nội


In [13]:
def quan_huyen_normalize(quan_huyen):
    quan_huyen = quan_huyen.lower()
    quan_huyen = spelling_correction(quan_huyen)
    if quan_huyen.startswith('quận'):
        quan_huyen_cut = quan_huyen[4:].strip()
        if 'từ liêm' in quan_huyen_cut:
            return '{}|{}'.format('từ liêm', quan_huyen_cut)
        if quan_huyen_cut.isdigit():
            return '{}|{}|{}'.format(quan_huyen, quan_huyen_cut, 'q'+quan_huyen_cut)
        return quan_huyen[4:].strip()
    elif quan_huyen.startswith('huyện'):
        return quan_huyen[5:].strip()
    elif quan_huyen.startswith('thành phố'):
        return quan_huyen[9:].strip()
    elif quan_huyen.startswith('thị xã'):
        return quan_huyen[6:].strip()
    else:
        return quan_huyen

In [14]:
df['Quận Huyện Normalized'] = df['Quận Huyện'].apply(quan_huyen_normalize)

In [15]:
df.head()

,Tỉnh Thành Phố,Mã TP,Quận Huyện,Mã QH,Phường Xã,Mã PX,Cấp,Tên Tiếng Anh,Tỉnh Thành Phố Normalized,Quận Huyện Normalized
0,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Phúc Xá,1,Phường,NaN,hà nội,ba đình
1,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Trúc Bạch,4,Phường,NaN,hà nội,ba đình
2,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Vĩnh Phúc,6,Phường,Vinh Phuc Commune,hà nội,ba đình
3,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Cống Vị,7,Phường,NaN,hà nội,ba đình
4,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Liễu Giai,8,Phường,Lieu Giai Commune,hà nội,ba đình


In [16]:
quan_huyen_normalized_set = set()

In [17]:
for index, row in df[['Quận Huyện Normalized', 'Tỉnh Thành Phố Normalized']].iterrows():
    quan_huyen_normalized_set.add((row[0], row[1]))

In [18]:
with open('./src/data/districts.txt', 'w') as f:
    for quan_huyen, tinh_tp in quan_huyen_normalized_set:
        f.write('{}\t{}\n'.format(quan_huyen, tinh_tp))

In [19]:
def phuong_xa_normalize(phuong_xa):
    phuong_xa = phuong_xa.lower()
    phuong_xa = spelling_correction(phuong_xa)
    phuong_xa = spelling_correction(phuong_xa)
    phuong_xa = ' '.join(phuong_xa.split())
    if phuong_xa.startswith('xã'):
        phuong_xa_cut = phuong_xa[2:]
    elif phuong_xa.startswith('phường'):
        phuong_xa_cut = phuong_xa[6:]
    elif phuong_xa.startswith('thị trấn'):
        phuong_xa_cut = phuong_xa[8:]
    else:
        phuong_xa_cut = phuong_xa
    phuong_xa_cut = phuong_xa_cut.strip()
    if len(phuong_xa_cut) < 5:
        if phuong_xa_cut.isdigit():
            if phuong_xa_cut.startswith('0'):
                phuong_xa_cut = phuong_xa_cut[1:]
            return '{}|{}|{}|{}'.format(phuong_xa, phuong_xa_cut, 'P'+phuong_xa_cut, 'phường ' + phuong_xa_cut)
        return '{}|{}'.format(phuong_xa, phuong_xa_cut)
    else:
        return phuong_xa_cut

In [20]:
df['Phường Xã Normalize'] = df['Phường Xã'].apply(phuong_xa_normalize)

In [21]:
df.head()

,Tỉnh Thành Phố,Mã TP,Quận Huyện,Mã QH,Phường Xã,Mã PX,Cấp,Tên Tiếng Anh,Tỉnh Thành Phố Normalized,Quận Huyện Normalized,Phường Xã Normalize
0,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Phúc Xá,1,Phường,NaN,hà nội,ba đình,phúc xá
1,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Trúc Bạch,4,Phường,NaN,hà nội,ba đình,trúc bạch
2,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Vĩnh Phúc,6,Phường,Vinh Phuc Commune,hà nội,ba đình,vĩnh phúc
3,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Cống Vị,7,Phường,NaN,hà nội,ba đình,cống vị
4,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Liễu Giai,8,Phường,Lieu Giai Commune,hà nội,ba đình,liễu giai


In [22]:
df[df['Quận Huyện Normalized'] == 'bình minh']

,Tỉnh Thành Phố,Mã TP,Quận Huyện,Mã QH,Phường Xã,Mã PX,Cấp,Tên Tiếng Anh,Tỉnh Thành Phố Normalized,Quận Huyện Normalized,Phường Xã Normalize
10249,Tỉnh Vĩnh Long,86,Thị xã Bình Minh,861,Phường Cái Vồn,29770,Phường,NaN,vĩnh long,bình minh,cái vồn
10250,Tỉnh Vĩnh Long,86,Thị xã Bình Minh,861,Phường Thành Phước,29771,Phường,NaN,vĩnh long,bình minh,thành phước
10251,Tỉnh Vĩnh Long,86,Thị xã Bình Minh,861,Xã Thuận An,29806,Xã,NaN,vĩnh long,bình minh,thuận an
10252,Tỉnh Vĩnh Long,86,Thị xã Bình Minh,861,Xã Đông Thạnh,29809,Xã,NaN,vĩnh long,bình minh,đông thạnh
10253,Tỉnh Vĩnh Long,86,Thị xã Bình Minh,861,Xã Đông Bình,29812,Xã,NaN,vĩnh long,bình minh,đông bình
10254,Tỉnh Vĩnh Long,86,Thị xã Bình Minh,861,Phường Đông Thuận,29813,Phường,NaN,vĩnh long,bình minh,đông thuận
10255,Tỉnh Vĩnh Long,86,Thị xã Bình Minh,861,Xã Mỹ Hòa,29815,Xã,NaN,vĩnh long,bình minh,mỹ hòa
10256,Tỉnh Vĩnh Long,86,Thị xã Bình Minh,861,Xã Đông Thành,29818,Xã,NaN,vĩnh long,bình minh,đông thành


In [23]:
phuong_xa_normalized_set = set()

In [24]:
for index, row in df[['Phường Xã Normalize','Quận Huyện Normalized', 'Tỉnh Thành Phố Normalized']].iterrows():
    phuong_xa_normalized_set.add((row[0], row[1], row[2]))

In [25]:
with open('./src/data/wards.txt', 'w') as f:
    for phuong_xa, quan_huyen, tinh_tp in phuong_xa_normalized_set:
        f.write('{}\t{}\t{}\n'.format(phuong_xa, quan_huyen, tinh_tp))
    f.write('{}\t{}\t{}\n'.format('nguyễn ư dĩ', 'hải dương', 'hải dương'))
    f.write('{}\t{}\t{}\n'.format('bùi thị xuân', 'hải dương', 'hải dương'))
    f.write('{}\t{}\t{}\n'.format('trần huy liệu', 'nam định', 'nam định'))
    f.write('{}\t{}\t{}\n'.format('lò đúc', 'hai bà trưng', 'hà nội'))        

In [26]:
phuong_xa_set = set(df['Phường Xã'])

In [27]:
phuong_xa_normalized_set = set()
for phuong_xa in phuong_xa_set:
#     l1 = len(phuong_xa_normalized_set)
    phuong_xa = phuong_xa.lower()
    if phuong_xa.startswith('xã'):
        phuong_xa_cut = phuong_xa[2:]
    elif phuong_xa.startswith('phường'):
        phuong_xa_cut = phuong_xa[6:]
    elif phuong_xa.startswith('thị trấn'):
        phuong_xa_cut = phuong_xa[8:]
    else:
        phuong_xa_cut = phuong_xa
    phuong_xa_cut = phuong_xa_cut.strip()
    if len(phuong_xa_cut) < 5:
        if not is_number(phuong_xa_cut):
            phuong_xa_normalized_set.add(phuong_xa_cut)
        phuong_xa_normalized_set.add(phuong_xa)
    else:
        phuong_xa_normalized_set.add(phuong_xa_cut)
#     l2 = len(phuong_xa_normalized_set)

NameError: name 'is_number' is not defined

In [28]:
len(address_entities)

NameError: name 'address_entities' is not defined

In [98]:
address_entities = phuong_xa_normalized_set | quan_huyen_normalized_set | tinh_tp_normalized_set

In [102]:
f = open('./adress_entities.txt', 'w')
for address_entity in address_entities:
    address_entity = ' '.join(address_entity.split())
    f.write(address_entity + '\n')
f.close()

In [107]:
import json

In [108]:
f = open('./json/11721197_01234518428_CMND_MatTruoc.json')
data = f.read()
f.close()

In [112]:
data = json.loads(data)

In [114]:
print(json.dumps(data, indent=4, ensure_ascii=False))

{
    "file": "11721197_01234518428_CMND_MatTruoc.jpg",
    "ID": [
        {
            "label": "022085002613",
            "box": {
                "name": "rect",
                "x": 357,
                "y": 137,
                "width": 336,
                "height": 39
            }
        }
    ],
    "name": [
        {
            "label": "TRẦN",
            "box": {
                "name": "rect",
                "x": 408,
                "y": 223,
                "width": 78,
                "height": 27
            }
        },
        {
            "label": "QUANG",
            "box": {
                "name": "rect",
                "x": 487,
                "y": 227,
                "width": 105,
                "height": 26
            }
        },
        {
            "label": "TỰ",
            "box": {
                "name": "rect",
                "x": 598,
                "y": 228,
                "width": 41,
                "height": 29
            }
      

In [116]:
for box in data['address']:
    print(box['label'])

Tổ
5
Tân
Lập
1
Cẩm
Thủy
Cẩm
Phả
Quảng
Ninh


# Build cost dictionary

In [29]:
a_chars = ['a', 'à', 'á', 'ạ', 'ã', 'ả']
aw_chars = ['ă', 'ằ', 'ắ', 'ặ', 'ẵ', 'ẳ']
aa_chars = ['â', 'ầ', 'ấ', 'ậ', 'ẫ', 'ẩ']
d_chars = ['d', 'đ']
e_chars = ['e', 'è', 'é', 'ẹ', 'ẽ', 'ẻ']
i_chars = ['i', 'ì', 'í', 'ị', 'ĩ', 'ỉ']
o_chars = ['o', 'ò', 'ó', 'ọ', 'õ', 'ỏ']
oo_chars = ['ô', 'ồ', 'ố', 'ộ', 'ỗ', 'ổ']
ow_chars = ['ơ', 'ờ', 'ớ', 'ợ', 'ỡ', 'ở']
u_chars = ['u', 'ù', 'ú', 'ụ', 'ũ', 'ủ']
uw_chars = ['ư', 'ừ', 'ứ', 'ự', 'ữ', 'ử']
y_chars = ['y', 'ỳ', 'ý', 'ỵ', 'ỹ', 'ỷ']
an_chars = ['a', 'n']
hl_chars = ['h', 'l']
# cách mnng tháng tám

In [30]:
similar_chars = [a_chars, aw_chars, aa_chars, d_chars, e_chars,
                 i_chars, o_chars, oo_chars, ow_chars, u_chars, 
                 uw_chars, y_chars, an_chars, hl_chars
                ]


In [33]:
f = open('./src/cost_char_dict.txt', 'w')
for similar_group in similar_chars:
    for i in range(len(similar_group)):
        for j in range(i+1, len(similar_group)):
            f.write('{}\t{}\t{}\n'.format(similar_group[i], similar_group[j], 0.3))
            f.write('{}\t{}\t{}\n'.format(similar_group[j], similar_group[i], 0.3))
def write_pairs(f, group1, group2, val):
    for e1 in group1:
        for e2 in group2:
            f.write('{}\t{}\t{}\n'.format(e1, e2, val))
write_pairs(f, a_chars, aw_chars, 0.4)
write_pairs(f, a_chars, aa_chars, 0.4)
write_pairs(f, aw_chars, aa_chars, 0.4)
write_pairs(f, a_chars, u_chars, 0.4)

write_pairs(f, ow_chars, o_chars, 0.4)
write_pairs(f, ow_chars, oo_chars, 0.4)
write_pairs(f, o_chars, oo_chars, 0.4)
f.close()

# Extract data from json raw

In [19]:
import os
import json
%matplotlib
import matplotlib.pyplot as plt
import imageio

Using matplotlib backend: Qt5Agg


In [3]:
data_path = '/media/trivu/data/DataScience/CV/CMT_data/raw/'

In [4]:
json_fn = os.listdir(data_path+'json')

In [22]:
f = open(data_path+'json/'+json_fn[3])

In [23]:
data = json.load(f)
f.close()

In [24]:
# !curl -X POST 10.3.9.222:5001 -F 'image=@/media/trivu/data/DataScience/CV/CMT_data/raw/images/841289701567_01203006157_341824958_1_2015101113241209.jpg'

In [25]:
data

{'file': '841289701567_01203007520_250871260_1_2015110219240343.jpg',
 'ID': [{'label': '250871260',
   'box': {'name': 'rect', 'x': 403, 'y': 120, 'width': 236, 'height': 37}}],
 'name': [{'label': 'HUỲNH',
   'box': {'name': 'rect', 'x': 364, 'y': 163, 'width': 149, 'height': 57}},
  {'label': 'MINH',
   'box': {'name': 'rect', 'x': 520, 'y': 165, 'width': 109, 'height': 52}},
  {'label': 'TUẤN',
   'box': {'name': 'rect', 'x': 631, 'y': 164, 'width': 119, 'height': 55}}],
 'DOB': [{'label': '25-04-1992',
   'box': {'name': 'rect', 'x': 483, 'y': 261, 'width': 194, 'height': 44}}],
 'address': [{'label': 'Đồng',
   'box': {'name': 'rect', 'x': 520, 'y': 313, 'width': 97, 'height': 47}},
  {'label': 'Nai',
   'box': {'name': 'rect', 'x': 617, 'y': 314, 'width': 68, 'height': 44}},
  {'label': 'Tổ',
   'box': {'name': 'rect', 'x': 595, 'y': 404, 'width': 53, 'height': 40}},
  {'label': '5B',
   'box': {'name': 'rect', 'x': 651, 'y': 398, 'width': 48, 'height': 47}},
  {'label': 'P',
  

In [27]:
img = imageio.imread('/media/trivu/data/DataScience/CV/CMT_data/raw/images/841289701567_01203007520_250871260_1_2015110219240343.jpg')

In [28]:
plt.imshow(img)